In [1]:
# parts of this code has been referenced from https://blog.quantinsti.com/vader-sentiment/
#import relevant libraries

# nltk for baseline unsupervised sentiment detection using vader lexicon
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# pandas and numpy for dealing with csv and numerical operations
import pandas as pd
import numpy as np
# for downloading the yahoo stocks data
import pandas_datareader as pdr
# requests to query newsapi
import requests

# Import BDay to determine business day's dates
from pandas.tseries.offsets import BDay

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/geshwar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# setting pandas options
pd.set_option('display.max_column', 500)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-2dadceb90db1>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
def get_stocks_data(stock, date):
    '''
    stock - AMD
    start date - 01-Oct-20
    '''
    # fetch data from yahoo
    data_amd = pdr.get_data_yahoo(stock, date) 
    
    # create buy and sell labels using sma - simple moving average
    data_amd['2_SMA'] = data_amd['Close'].rolling(window=2).mean()
    data_amd['5_SMA'] = data_amd['Close'].rolling(window=5).mean()

    data_amd = data_amd[data_amd['5_SMA'].notna()]

    # SMA trade calls
    Trade_Buy=[]
    Trade_Sell=[]
    labels = []
    for i in range(len(data_amd)-1):
        if ((data_amd['2_SMA'].values[i] < data_amd['5_SMA'].values[i]) & (data_amd['2_SMA'].values[i+1] > data_amd['5_SMA'].values[i+1])):
            #print("Trade Call for {row} is Buy.".format(row=data_amd.index[i].date()))
            Trade_Buy.append(i)
            labels.append('Buy')
        elif ((data_amd['2_SMA'].values[i] > data_amd['5_SMA'].values[i]) & (data_amd['2_SMA'].values[i+1] < data_amd['5_SMA'].values[i+1])):
            Trade_Sell.append(i)
            labels.append('Sell')
        else:
            labels.append('Hold')
    labels.append('Hold')
    data_amd['label'] = labels
    data_amd['ticker'] = [stock]*data_amd.shape[0]
    return data_amd

def get_trade_open(date):
    #print(date, type(date))
    curr_date_open = pd.to_datetime(date).floor('d').replace(hour=13,minute=30) - BDay(0)
    curr_date_close = pd.to_datetime(date).floor('d').replace(hour=20,minute=0) - BDay(0)
    
    prev_date_close = (curr_date_open - BDay()).replace(hour=20,minute=0)
    next_date_open = (curr_date_close + BDay()).replace(hour=13,minute=30)
    
    if ((pd.to_datetime(date)>=prev_date_close) & (pd.to_datetime(date)<curr_date_open)):
        return curr_date_open
    elif ((pd.to_datetime(date)>=curr_date_close) & (pd.to_datetime(date)<next_date_open)):
        return next_date_open
    else:
        return None

def get_news_data(stock):
    url = 'https://newsapi.org/v2/everything?'

    # Specify the query and number of returns
    parameters = {
        'q': stock, # query phrase
        'sortBy': 'popularity', # articles from popular sources and publishers come first
        'pageSize': 100,  # maximum is 100 for developer version
        'apiKey': 'f5b81eee26704785a438a179f2d88f2d', # your own API key
        'from':'2020-10-17',
        'to':'2020-11-17'
    }

    # Make the request
    response = requests.get(url, params=parameters)

    # Convert the response to JSON format and store it in dataframe
    data = pd.DataFrame(response.json())

    news_df = pd.concat([data['articles'].apply(pd.Series)], axis=1)
    final_news = news_df.loc[:,['publishedAt','title', 'description']]
    final_news['publishedAt'] = pd.to_datetime(final_news['publishedAt'])
    final_news.sort_values(by='publishedAt',inplace=True)
    
    final_news["trading_time"] = final_news["publishedAt"].apply(get_trade_open)

    final_news = final_news[pd.notnull(final_news['trading_time'])]
    final_news['Date'] = pd.to_datetime(pd.to_datetime(final_news['trading_time']).dt.date)
    print(final_news.shape)
    
    # merging the news title and description for same day
    title_concat = final_news.groupby(['Date'])['title'].apply(lambda x: '. '.join(x)).reset_index()
    description_concat = final_news.groupby(['Date'])['description'].apply(lambda x: '. '.join(x)).reset_index()

    final_text = title_concat.merge(description_concat, left_on='Date', right_on='Date')

    return final_text

def get_full_data():
    
    stock_list = ['AMZN', 'GOOGL', 'MSFT', 'AAPL', 'TSLA', 'NFLX', 'DIS', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EVRG', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FLIR', 'FLS', 'FMC', 'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS', 'GWW', 'HAL', 'HBI', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HFC', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'ZM', 'TWTR', 'SNAP', 'AMD']
    df = pd.DataFrame()
    for st in stock_list:
        print("Stock", st)
        stock_data = get_stocks_data(st, '16-Oct-20')
        try:
            text = get_news_data(st)
        except Exception as e:
            print(" failed for stock ", st, e)
            continue
        final_merge = text.merge(stock_data, left_on='Date', right_on='Date')
        print("final shape", final_merge.shape)
        
        # vader sentiment detector
        analyzer = SentimentIntensityAnalyzer()

        cs = []
        final_merge['text'] = final_merge['title'] + '. '+ final_merge['description']
        for row in range(len(final_merge)):
            cs.append(analyzer.polarity_scores(final_merge['text'].iloc[row])['compound'])

        final_merge['compound_vader_score'] = cs
        
        df = pd.concat([df, final_merge], ignore_index=True)
    return df

final_df = get_full_data()
final_df.to_csv('stock-data-with-sentiment.csv')

Stock AMZN
(57, 5)
final shape (20, 13)
Stock GOOGL
(57, 5)
final shape (18, 13)
Stock MSFT
(61, 5)
final shape (18, 13)
Stock AAPL
(55, 5)
final shape (20, 13)
Stock TSLA
(56, 5)
final shape (16, 13)
Stock NFLX
(64, 5)
final shape (17, 13)
Stock DIS
(66, 5)
final shape (17, 13)
Stock ETN
(52, 5)
final shape (16, 13)
Stock EBAY
 failed for stock  EBAY If using all scalar values, you must pass an index
Stock ECL
(64, 5)
 failed for stock  ECL sequence item 1: expected str instance, NoneType found
Stock EIX
(41, 5)
final shape (13, 13)
Stock EW
(64, 5)
final shape (18, 13)
Stock EA
(50, 5)
final shape (15, 13)
Stock EMR
(63, 5)
final shape (19, 13)
Stock ETR
(78, 5)
final shape (17, 13)
Stock EOG
(58, 5)
final shape (18, 13)
Stock EFX
(61, 5)
final shape (18, 13)
Stock EQIX
(37, 5)
final shape (12, 13)
Stock EQR
(33, 5)
final shape (16, 13)
Stock ESS
(66, 5)
final shape (19, 13)
Stock EL
(73, 5)
final shape (19, 13)
Stock ETSY
(50, 5)
final shape (16, 13)
Stock EVRG
(11, 5)
final shape (

In [9]:
final_df

,Date,title,description,High,Low,Open,Close,Volume,Adj Close,2_SMA,5_SMA,label,ticker,text,compound_vader_score
0,2020-10-21,"The 10 Best Deals of October 20, 2020",Tuesday’s Best Deals | Kinja DealsRead more...,81.930000,79.190002,81.720001,79.199997,36426400,79.199997,80.379997,81.811998,Hold,AMD,"The 10 Best Deals of October 20, 2020. Tuesday’s Best Deals | Kinja DealsRead more...",0.8555
1,2020-10-22,Acer announces several new Intel-powered laptop lines,Acer appears to have gone all-in on Intel's Tiger Lake CPU lineup.,80.820000,78.419998,79.650002,79.419998,38006800,79.419998,79.309998,81.069998,Hold,AMD,Acer announces several new Intel-powered laptop lines. Acer appears to have gone all-in on Intel's Tiger Lake CPU lineup.,0.0000
2,2020-10-23,"Intel raises annual sales, profit forecast - Reuters. Intel's revenue from data-center business misses estimates, shares slide - Reuters UK. UPDATE 1-Intel data center revenue misses estimates, shares slide - Reuters India. Intel's revenue from data-center business misses estimates, shares slide - Reuters India. Intel CEO expresses optimism even as stock falls after Q3 earnings. Intel's margins tumble as customers shift to cheaper chips, shares slide 10% - Reuters India. How to try out Microsoft Edge right now on a Chromebook","Intel Corp <INTC.O> missed revenue expectations for its high-margin data center business on Thursday, sending its shares down 8% in trading after the bell.. Intel Corp <INTC.O> on Thursday missed revenue estimates for its data-center business for the third quarter as the COVID-19 pandemic pushed its government and business customers to spend less on its chips, sending its shares down 8%.. Intel Corp missed revenue expectations for its high-margin data center business on Thursday, sending its shares down 8% in trading after the bell.. Intel Corp <INTC.O> on Thursday missed revenue estimates for its data-center business for the third quarter as the COVID-19 pandemic pushed its government and business customers to spend less on its chips, sending its shares down 8%.. Intel's earnings prompted a selloff, but in a conference call with analysts, CEO Bob Swan expressed optimism about the future.. Intel Corp on Thursday reported that margins tumbled in the latest quarter as consumers bought cheaper laptops and pandemic-stricken businesses and governments clamped down on data center spending, news that sent its shares down 10%.. Microsoft just launched the first version of the new Edge browser for Linux, which means you can also use it on a Chromebook. Because, why not?\n\nNow that Microsoft has finally released Edge for Linux, albeit in preview, the three traditional major desktop pla…",81.989998,79.330002,80.930000,81.959999,46557700,81.959999,80.689999,80.827998,Buy,AMD,"Intel raises annual sales, profit forecast - Reuters. Intel's revenue from data-center business misses estimates, shares slide - Reuters UK. UPDATE 1-Intel data center revenue misses estimates, shares slide - Reuters India. Intel's revenue from data-center business misses estimates, shares slide - Reuters India. Intel CEO expresses optimism even as stock falls after Q3 earnings. Intel's margins tumble as customers shift to cheaper chips, shares slide 10% - Reuters India. How to try out Microsoft Edge right now on a Chromebook. Intel Corp <INTC.O> missed revenue expectations for its high-margin data center business on Thursday, sending its shares down 8% in trading after the bell.. Intel Corp <INTC.O> on Thursday missed revenue estimates for its data-center business for the third quarter as the COVID-19 pandemic pushed its government and business customers to spend less on its chips, sending its shares down 8%.. Intel Corp missed revenue expectations for its high-margin data center business on Thursday, sending its shares down 8% in trading after the bell.. Intel Corp <INTC.O> on Thursday missed revenue estimates for its data-center business for the third quarter as the COVID-19 pandemic pushed its

In [25]:
url = 'https://newsapi.org/v2/everything?'

# Specify the query and number of returns
parameters = {
    'q': 'AMD', # query phrase
    'sortBy': 'popularity', # articles from popular sources and publishers come first
    'pageSize': 100,  # maximum is 100 for developer version
    'apiKey': 'f5b81eee26704785a438a179f2d88f2d', # your own API key
    'from':'2020-10-16',
    'to':'2020-11-16'
}

# Make the request
response = requests.get(url, params=parameters)

In [26]:
response.json()

{'status': 'error',
 'code': 'rateLimited',
 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [27]:
final_df.shape

(983, 15)